In [115]:
# Importazione delle librerie necessarie
import os
import logging
from io import BytesIO
import tempfile
import asyncio
import re
import zipfile
import PyPDF2
from docx import Document
from dotenv import load_dotenv
import edge_tts
# Importazioni specifiche per la gestione dell'API OpenAI e del modello
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate



In [117]:
# Configurazione del logging per tracciare le operazioni durante l'esecuzione
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Messaggio di test per verificare il funzionamento del logging
logger.info("Logging configurato correttamente.")


2024-10-25 13:37:56,197 - INFO - Logging configurato correttamente.


In [118]:
# Load environment variables
load_dotenv()

True

In [121]:
# Dizionario delle lingue e delle voci disponibili per la sintesi vocale
language_dict = {
    "Italian": {
        "Isabella": "it-IT-IsabellaNeural",
        "Diego": "it-IT-DiegoNeural",
        "Elsa": "it-IT-ElsaNeural"
    },
    "English": {
        "Jenny": "en-US-JennyNeural",
        "Guy": "en-US-GuyNeural",
        "Aria": "en-US-AriaNeural"
    },
    "Swedish": {
        "Sofie": "sv-SE-SofieNeural",
        "Mattias": "sv-SE-MattiasNeural"
    }
}

# Lista dei modelli LLM disponibili
llm_models = [
    "gpt-4o-mini", # Modello leggero per riassunti rapidi
    "gpt-4o",      # Modello avanzato per riassunti dettagliati
    "gpt-4"        # Modello per task complessi 
]

# Impostazioni configurabili per il modello e la sintesi vocale
DEFAULT_LANGUAGE = "Italian"       # Lingua di default per il riassunto e la sintesi vocale
DEFAULT_SPEAKER = "Isabella"       # Voce di default per la sintesi vocale in italiano
DEFAULT_MODEL = llm_models[0]      # Modello LLM di default per il riassunto
DEFAULT_TEMPERATURE = 0.7          # Temperatura di default per il modello

# Funzione per configurare e restituire l'istanza del modello LLM
def configure_llm():
    """Configura e restituisce l'istanza del modello LLM con le impostazioni di default."""
    llm = ChatOpenAI(
        model=DEFAULT_MODEL,
        temperature=DEFAULT_TEMPERATURE,
        openai_api_key=openai_api_key  # La chiave API viene caricata dall'ambiente .env
    )
    return llm



In [122]:
file_path="/Users/desi76/EduRag/EduRag/app/Art_Competenze strategiche, prospettiva temporale e dimensione narrativa nell’orientamento.pdf"

In [75]:
def extract_text_from_pdf(file_path):
    """Estrae il testo da un file PDF."""
    text = ""
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n\n"
    return text


In [123]:
def extract_text_from_docx(file_path):
    """Estrae il testo da un file DOCX."""
    doc = Document(file_path)
    text = "\n\n".join([para.text for para in doc.paragraphs])
    return text


In [124]:
def extract_text_from_txt(file_path):
    """Estrae il testo da un file TXT."""
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    return text


In [125]:
def load_and_extract_text(file_path):
    """Carica ed estrae il testo da un file PDF, DOCX o TXT senza pulizia della formattazione Markdown."""
    file_type = file_path.split('.')[-1].lower()
    
    if file_type == "pdf":
        text = extract_text_from_pdf(file_path)
    elif file_type == "docx":
        text = extract_text_from_docx(file_path)
    elif file_type == "txt":
        text = extract_text_from_txt(file_path)
    else:
        raise ValueError("Formato file non supportato.")
    
    # Restituisce il testo senza pulizia della formattazione Markdown
    return text



In [126]:
# Carica ed estrai il testo
testo_caricato = load_and_extract_text(file_path)

# Visualizza l'output
print("Testo caricato:")
print("  ")
print(testo_caricato)

Testo caricato:
  
43 Competenze strategiche, prospettiva temporale e dimensione 
narrativa nell’orientamento 
Strategic skills, time perspective and narrative dimension 
in educational guidance 
Nel presente lavoro sono riportati gli esiti 
dell’applicazione, a studenti universitari, di 
strumenti quali il Questionario sulle Strate -
gie di Apprendimento (QSA-Pellerey, 1996),
il Questionario di Percezione delle Compe -
tenze Strategiche (QPCS-Pellerey, 2010) e lo 
Zimbardo Time Perspective Inventory (ZTPI- 
Zimbardo e Boyd, 1999). È stato rilevato che 
il possesso di competenze strategiche ade -
guate correla positivamente con una pro -
spettiva temporale equilibrata e viceversa gli 
studenti con competenze strategiche inade -
guate mostrano un vissuto “fatalista” del 
presente e scarso orientamento nel futuro.
Ciò ha indotto l’adozione di una prospettiva 
diacronico-narrativa per l’orientamento con 
la proposta di redazione di un ePortfolio 
(Barrett, 2004) in cui convergano le rifle

In [127]:
def create_outline_from_text(text, llm):
    """Genera e restituisce un outline dell'intero testo senza suddivisione in chunk, utilizzando un oggetto LLM preconfigurato."""
    
    template = ChatPromptTemplate.from_messages([
        ("system", f"""
        Il tuo compito è creare un outline dettagliato del seguente testo in {DEFAULT_LANGUAGE}.
        
        L'outline deve essere ben strutturato e organizzato gerarchicamente con i seguenti requisiti:
        - Inizia ogni sezione con un titolo chiaro e conciso che riassuma il contenuto della sezione.
        - Se necessario, crea sottosezioni per riflettere la struttura interna del testo.
        - Evita duplicazioni o ripetizioni tra le sezioni e mantieni solo le idee principali.
        - Mantieni l'ordine logico delle idee e la coerenza tra le sezioni.
        - Se il testo contiene esempi o dettagli aggiuntivi, segnalali chiaramente ma in modo conciso.
        
        Fornisci l'output in forma di elenco strutturato:
        - Usa numeri per le sezioni principali (1, 2, 3, ...)
        - Usa sotto-numeri per le sottosezioni (1.1, 1.2, ...)
        
        Testo per l'outline:
        """),
        ("human", "{input}")
    ])
    
    chain = template.pipe(llm)
    response = chain.invoke({
        "input": text
    })
    
    return response.content







In [128]:
# Configura l'oggetto LLM una sola volta
llm = configure_llm()

# Esegui la funzione per creare l'outline utilizzando l'oggetto llm configurato
outline = create_outline_from_text(testo_caricato, llm)

# Stampa l'outline generato
print("Outline Generato:")
print(outline)



2024-10-25 13:39:23,238 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Outline Generato:
1. Introduzione
   1.1 Scopo del lavoro
   1.2 Strumenti utilizzati
      - Questionario sulle Strategie di Apprendimento (QSA)
      - Questionario di Percezione delle Competenze Strategiche (QPCS)
      - Zimbardo Time Perspective Inventory (ZTPI)
   1.3 Risultati preliminari
      - Correlazione tra competenze strategiche e prospettiva temporale

2. Competenze strategiche, prospettiva temporale e motivazione
   2.1 Importanza dell'orientamento
      - Criticità nei sistemi formativi
      - Rilevanza delle prospettive per l'autonomia
   2.2 Teorie motivazionali
      - Motivazione intrinseca ed estrinseca
      - Autodeterminazione e competenza
   2.3 Autoregolazione dell'apprendimento
      - Descrizione di una persona autoregolata
      - Competenze strategiche per l'autoregolazione
   2.4 Motivazione e prospettiva temporale
      - Legame tra motivazione e capacità di pianificazione
      - Ricerche sulla prospettiva temporale e successo scolastico

3. Laborator

In [129]:
def summarize_chunk_with_context(chunk, prev_chunk, next_chunk, llm):
    """Riassume un chunk di testo, tenendo conto del chunk precedente e successivo e stampa il risultato."""
    
    template = ChatPromptTemplate.from_messages([
        ("system", f"""
        Il tuo compito è creare un riassunto chiaro e coeso per un segmento di testo in {DEFAULT_LANGUAGE}.
        
        Segui questi requisiti:
        - Riepiloga solo le idee centrali e i punti principali del chunk corrente, senza aggiungere dettagli superflui.
        - Mantieni continuità e coerenza logica rispetto ai chunk precedente e successivo.
        - Usa frasi brevi e precise per esprimere chiaramente i concetti chiave.
        
        Contesto:
        - Chunk precedente (fornisce il contesto): "{prev_chunk}"
        - Chunk corrente (il testo da riassumere): "{chunk}"
        - Chunk successivo (contesto per continuità): "{next_chunk}"

        Il riassunto deve essere in un breve paragrafo (2-3 frasi), e mantenere un flusso naturale e coeso con il resto del testo.
        """),
        ("human", "{input}")
    ])
    
    chain = template.pipe(llm)
    response = chain.invoke({
        "input": chunk  # Passa solo il chunk corrente per il riassunto
    })
    
    summary = response.content
    print("Riassunto Generato per il Chunk:")
    print(summary)
    
    return summary

In [130]:
def split_text_by_paragraphs(text):
    """Divide il testo in blocchi basati sui paragrafi."""
    paragraphs = text.split('\n\n')  # Divide il testo per paragrafi
    chunks = [para.strip() for para in paragraphs if para.strip()]  # Rimuove spazi vuoti extra
    return chunks



In [131]:
# Divide il testo in chunk basati sui paragrafi
semantic_chunks = split_text_by_paragraphs(testo_caricato)

# Visualizza i chunk generati
for i, chunk in enumerate(semantic_chunks, start=1):
    print(f"\nChunk {i}:\n{chunk}\n{'-'*40}")



Chunk 1:
43 Competenze strategiche, prospettiva temporale e dimensione 
narrativa nell’orientamento 
Strategic skills, time perspective and narrative dimension 
in educational guidance 
Nel presente lavoro sono riportati gli esiti 
dell’applicazione, a studenti universitari, di 
strumenti quali il Questionario sulle Strate -
gie di Apprendimento (QSA-Pellerey, 1996),
il Questionario di Percezione delle Compe -
tenze Strategiche (QPCS-Pellerey, 2010) e lo 
Zimbardo Time Perspective Inventory (ZTPI- 
Zimbardo e Boyd, 1999). È stato rilevato che 
il possesso di competenze strategiche ade -
guate correla positivamente con una pro -
spettiva temporale equilibrata e viceversa gli 
studenti con competenze strategiche inade -
guate mostrano un vissuto “fatalista” del 
presente e scarso orientamento nel futuro.
Ciò ha indotto l’adozione di una prospettiva 
diacronico-narrativa per l’orientamento con 
la proposta di redazione di un ePortfolio 
(Barrett, 2004) in cui convergano le riflessio -
ni

In [132]:
# Configura l'oggetto LLM una sola volta
llm = configure_llm()

# Lista per conservare i riassunti di ciascun chunk
summaries = []

# Riassumi ciascun chunk utilizzando il contesto dei chunk adiacenti
for i, chunk in enumerate(semantic_chunks):
    prev_chunk = semantic_chunks[i - 1] if i > 0 else ""
    next_chunk = semantic_chunks[i + 1] if i < len(semantic_chunks) - 1 else ""
    summary = summarize_chunk_with_context(chunk, prev_chunk, next_chunk, llm)
    summaries.append(summary)

# Stampa ciascun riassunto per verificare il risultato
for i, summary in enumerate(summaries, start=1):
    print(f"Riassunto del chunk {i}:\n{summary}\n{'-'*40}")


2024-10-25 13:39:50,319 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Riassunto Generato per il Chunk:
Il lavoro presenta i risultati di un’indagine condotta su studenti universitari utilizzando tre strumenti: il Questionario sulle Strategie di Apprendimento, il Questionario di Percezione delle Competenze Strategiche e lo Zimbardo Time Perspective Inventory. Si evidenzia che competenze strategiche adeguate sono correlate a una prospettiva temporale equilibrata, mentre competenze inadeguate portano a una visione fatalista del presente e a scarso orientamento verso il futuro. Di conseguenza, è stata proposta la creazione di un ePortfolio per facilitare la reinterpretazione delle esperienze passate e la proiezione nel futuro.


2024-10-25 13:39:54,648 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Riassunto Generato per il Chunk:
Negli ultimi decenni, l'orientamento scolastico e professionale ha assunto un'importanza strategica per affrontare le criticità dei sistemi formativi, come la dispersione scolastica e l'incertezza futura. Le teorie di orientamento enfatizzano l'autonomia e l'autodeterminazione, promuovendo una prospettiva diacronica che integra la dimensione narrativa per dare significato ai progetti di vita. Studi sulla motivazione evidenziano l'importanza di comportamenti orientati all'esplorazione e alla padronanza, collegando la percezione del passato con la motivazione e il raggiungimento di obiettivi futuri.


2024-10-25 13:39:59,242 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Riassunto Generato per il Chunk:
Il segmento discute vari studi e pubblicazioni riguardanti l'orientamento educativo e l'uso degli e-portfolios per valutare l'apprendimento degli studenti. Si sottolinea l'importanza della prospettiva temporale e delle competenze strategiche nel contesto universitario, evidenziando come questi elementi influenzino la motivazione e l'impegno accademico. Inoltre, vengono citati diversi autori e ricerche che esplorano la relazione tra motivazione, apprendimento e gestione del tempo nella formazione degli studenti.


2024-10-25 13:40:41,155 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Riassunto Generato per il Chunk:
Il segmento discute diversi lavori di Pellerey riguardanti le competenze strategiche, il loro sviluppo e valutazione, esplorando anche l'orientamento professionale e la prospettiva esistenziale. Vengono presentati strumenti per l'autovalutazione e l'insegnamento delle competenze strategiche, insieme a ricerche su come le prospettive temporali influenzano l'apprendimento e l'autoregolazione degli studenti. Inoltre, il testo menziona studi che collegano l'orientamento al futuro con obiettivi e strategie di apprendimento.


2024-10-25 13:40:44,186 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Riassunto Generato per il Chunk:
Il segmento presenta una serie di riferimenti bibliografici riguardanti la motivazione, l'apprendimento e la gestione del tempo nell'ambito educativo. Vengono citati autori significativi come Toffler, Weiner e Zimbardo, i quali esplorano tematiche come il futuro, le strategie di valutazione e l'autoregolazione nell'apprendimento. Queste opere contribuiscono a comprendere le dinamiche tra motivazione, apprendimento e successo accademico.
Riassunto del chunk 1:
Il lavoro presenta i risultati di un’indagine condotta su studenti universitari utilizzando tre strumenti: il Questionario sulle Strategie di Apprendimento, il Questionario di Percezione delle Competenze Strategiche e lo Zimbardo Time Perspective Inventory. Si evidenzia che competenze strategiche adeguate sono correlate a una prospettiva temporale equilibrata, mentre competenze inadeguate portano a una visione fatalista del presente e a scarso orientamento verso il futuro. Di conseguenza, è stata p

In [133]:
def combine_summaries(summaries):
    """Unisce i riassunti dei chunk in un unico testo e lo stampa."""
    combined_text = "\n\n".join(summaries)
    print("Testo combinato da tutti i chunk riassunti:")
    print(combined_text)
    return combined_text


In [134]:
# Combina i riassunti in un unico testo
testo_combinato = combine_summaries(summaries)


Testo combinato da tutti i chunk riassunti:
Il lavoro presenta i risultati di un’indagine condotta su studenti universitari utilizzando tre strumenti: il Questionario sulle Strategie di Apprendimento, il Questionario di Percezione delle Competenze Strategiche e lo Zimbardo Time Perspective Inventory. Si evidenzia che competenze strategiche adeguate sono correlate a una prospettiva temporale equilibrata, mentre competenze inadeguate portano a una visione fatalista del presente e a scarso orientamento verso il futuro. Di conseguenza, è stata proposta la creazione di un ePortfolio per facilitare la reinterpretazione delle esperienze passate e la proiezione nel futuro.

Negli ultimi decenni, l'orientamento scolastico e professionale ha assunto un'importanza strategica per affrontare le criticità dei sistemi formativi, come la dispersione scolastica e l'incertezza futura. Le teorie di orientamento enfatizzano l'autonomia e l'autodeterminazione, promuovendo una prospettiva diacronica che int

In [137]:
def revise_text_with_outline(text, outline, llm):
    """Usa l'LLM per rivedere e ristrutturare il testo combinato seguendo l'outline generato."""
    
    template = ChatPromptTemplate.from_messages([
        ("system", f"""
        Il tuo compito è rivedere e ristrutturare il testo seguente per renderlo coerente e scorrevole in {DEFAULT_LANGUAGE}.
        
        Requisiti per la revisione:
        - Segui la struttura dell'outline fornito per mantenere un ordine logico delle idee.
        - Integra le sezioni in modo naturale per evitare ripetizioni.
        - Migliora la scorrevolezza e coerenza del testo, mantenendo solo i punti chiave.
        - Mantieni il tono formale e informativo.
        
        Outline fornito:
        {outline}
        
        Testo da rivedere:
        """),
        ("human", "{input}")
    ])
    
    chain = template.pipe(llm)
    response = chain.invoke({
        "input": text  # Usa il testo combinato
    })
    
    revised_text = response.content
    print("Testo revisionato e coerente:")
    return revised_text


In [138]:
# Esegui la revisione del testo combinato usando l'outline come guida
testo_finale = revise_text_with_outline(testo_combinato, outline, llm)

# Visualizza il testo finale coerente
print("Testo finale coerente e revisionato:")
print(testo_finale)


2024-10-25 13:46:53,573 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Testo revisionato e coerente:
Testo finale coerente e revisionato:
**1. Introduzione**

Il presente lavoro si propone di analizzare i risultati di un’indagine condotta su studenti universitari, utilizzando tre strumenti di valutazione: il Questionario sulle Strategie di Apprendimento (QSA), il Questionario di Percezione delle Competenze Strategiche (QPCS) e lo Zimbardo Time Perspective Inventory (ZTPI). L'obiettivo principale è evidenziare la correlazione tra competenze strategiche adeguate e una prospettiva temporale equilibrata, rispetto a competenze inadeguate che tendono a generare una visione fatalista del presente e una scarsa orientazione verso il futuro. A tal fine, si propone la creazione di un ePortfolio per facilitare la reinterpretazione delle esperienze passate e la proiezione nel futuro.

**2. Competenze strategiche, prospettiva temporale e motivazione**

Negli ultimi decenni, l'orientamento scolastico e professionale ha acquisito un'importanza strategica per affrontare l

In [142]:
import re
from docx import Document

def apply_text_formatting(paragraph, text):
    """
    Applica la formattazione di grassetto e corsivo al testo all'interno del paragrafo di Word.
    """
    # Gestione del grassetto e del corsivo combinati: ***testo***
    matches = re.findall(r'\*\*\*(.*?)\*\*\*', text)
    for match in matches:
        run = paragraph.add_run(match)
        run.bold = True
        run.italic = True
        text = text.replace(f'***{match}***', '')

    # Gestione del grassetto: **testo**
    matches = re.findall(r'\*\*(.*?)\*\*', text)
    for match in matches:
        run = paragraph.add_run(match)
        run.bold = True
        text = text.replace(f'**{match}**', '')

    # Gestione del corsivo: *testo*
    matches = re.findall(r'\*(.*?)\*', text)
    for match in matches:
        run = paragraph.add_run(match)
        run.italic = True
        text = text.replace(f'*{match}*', '')

    # Aggiungi il testo rimanente senza formattazione
    paragraph.add_run(text)

def export_to_word(text, filename="testo_finale.docx"):
    """Esporta il testo finale in un file Word (.docx) con formattazione basata sui marcatori Markdown."""
    doc = Document()
    
    for line in text.splitlines():
        line = line.strip()
        if line.startswith("### "):  # Titolo di terzo livello
            heading = doc.add_heading(level=3)
            apply_text_formatting(heading, line[4:])
        elif line.startswith("## "):  # Titolo di secondo livello
            heading = doc.add_heading(level=2)
            apply_text_formatting(heading, line[3:])
        elif line.startswith("# "):  # Titolo di primo livello
            heading = doc.add_heading(level=1)
            apply_text_formatting(heading, line[2:])
        else:
            # Testo normale con possibile formattazione Markdown (grassetto, corsivo)
            paragraph = doc.add_paragraph()
            apply_text_formatting(paragraph, line)
    
    doc.save(filename)
    print(f"Testo esportato in formato Word con formattazione: {filename}")



In [143]:
def export_to_txt(text, filename="testo_finale.txt"):
    """Esporta il testo finale in un file di testo semplice (.txt)."""
    with open(filename, "w", encoding="utf-8") as file:
        file.write(text)
    print(f"Testo esportato in formato TXT: {filename}")



In [144]:
# Esporta il testo finale in Word e TXT
export_to_word(testo_finale, "testo_finale.docx")
export_to_txt(testo_finale, "testo_finale.txt")



Testo esportato in formato Word con formattazione: testo_finale.docx
Testo esportato in formato TXT: testo_finale.txt


In [149]:
import nest_asyncio
nest_asyncio.apply()


In [150]:
async def save_audio_from_text_file(file_path="testo_finale.txt", output_audio="testo_finale.mp3"):
    """Converte il testo da un file .txt in audio e salva il file usando il servizio Edge Text-to-Speech con le configurazioni di default."""
    
    # Leggi il contenuto del file di testo
    with open(file_path, "r", encoding="utf-8") as file:
        audio_text = file.read()
    
    # Ottiene la voce specifica dal dizionario per la lingua e speaker di default
    voice = language_dict[DEFAULT_LANGUAGE][DEFAULT_SPEAKER]
    
    # Genera il file audio utilizzando la funzione text_to_speech_edge_async
    audio_path = await text_to_speech_edge_async(audio_text, DEFAULT_LANGUAGE, DEFAULT_SPEAKER)
    
    # Leggi l’audio generato e salvalo nel formato desiderato
    with open(audio_path, "rb") as audio_file:
        audio_data = audio_file.read()
        
    # Salva l’audio in formato MP3
    with open(output_audio, "wb") as output_file:
        output_file.write(audio_data)
    
    print(f"Audio generato e salvato come: {output_audio}")


In [151]:
# Esegui la funzione asincrona per generare e salvare l’audio
await save_audio_from_text_file(file_path="testo_finale.txt", output_audio="testo_finale.mp3")


Audio generato e salvato come: testo_finale.mp3
